# How to use the FormationEnergyQE Workchain

This notebook will explain the use of the `FormationEnergyQE` workchain. 

This workchain implements the computation of the formation energy of a defect using QuantumESPRESSO.
This is one typical use-case of `AiiDA-Defects` and might be used in the situation in which you already have a defect structure you wish to work with. Note that there is also a `FormationEnergyBase` workchain, but this workchain cannot be used directly. One should always used one of the workchains dervived for a particular simulation code, such as `FormationEnergyQE`.

In example below, the `GaussianCountercharge` correction workchain is used as the electrostatic correction. After specifying some details about the defect of interest and making some QuantumESPRESSO specific choices, the `FormationEnergyQE` workchain will complete the following steps:
1. Run self-consistent DFT energy calculations for the host, neutral defect and charged defect supercells.
2. Run PP.x to obtain the electrostatic potentials.
3. Run a further DFT calculation on a unitcell (optionally).
4. Run DFPT to obtain the relative permitivitty (optionally).
5. Run the `GaussianCountercharge` workchain to obtain the correction.
6. Compute the formation energy, with and without corrections.

## Step-by-step Usage 
1. Set up AiiDA as normal

In [ ]:
# Get your normal profile
%load_ext aiida

In [ ]:
%aiida

In [ ]:
# Import commonly used functionality
import numpy as np
from aiida import orm, engine, common

2. Import the workchain. Any issues with the import, such as the entry point not being found, would indicate an installation problem.

In [ ]:
from aiida_defects.formation_energy.formation_energy_qe import FormationEnergyWorkchainQE
from aiida_defects.formation_energy.utils import generate_defect_structure

3. Prepare some structures

In [ ]:
# Set up structures
from pymatgen.core.structure import Structure

pymatgen_structure = Structure.from_file("./Structures/Li3ClO_1x1x1.cif")

# Unitcell
unitcell_structure = orm.StructureData(pymatgen=pymatgen_structure)

# Host 2x2x2 supercell
pymatgen_structure.make_supercell([2,2,2])
host_structure = orm.StructureData(pymatgen=pymatgen_structure)

# Defect (Lithium vacancy) 2x2x2 supercell
defect_position = [0.0, 0.0, 0.0]
defect_structure = generate_defect_structure(host_structure, defect_position, {'Li': -1}) # the value -1 means removing (to create vacancy) while +1 mean adding (to create interstitial)

4. Prepare the inputs for the workchain. A builder can be used (as per the `GaussianCountercharge` example) or the input dictionary can be populated directly as shown below.

4a.  Set up the supercell calculations. Most of the parameters needed for the pw calculations are taken from the aiida-quantum espresso protocol but the users can also overwrite these default parameters by their owns.

In [ ]:
pw_code = orm.Code.get_from_string('pw@localhost')

# set this parameters to True if you start the calculations from scratch
run_pw = True
run_v = True
run_rho = True

# Pseudos 
pseudo_family = orm.Str('SSSP/1.2/PBEsol/efficiency') # This is the label that was used when installing the pseudos

# Scheduler options
pw_metadata = orm.Dict( dict={
    'description': 'Li3ClO test', 
    'options': {
        'max_wallclock_seconds': 1800, 
        'resources': {
            'num_machines': 1
        }
    }, 
    'label': 'Li3ClO test'
})

4b. Set up the post-processing calculations. These are used to obtain the electrostatic potentials from the supercell calculations.

In [ ]:
pp_code = orm.Code.get_from_string('pp@localhost')

# Scheduler options
pp_metadata = orm.Dict( dict={
    'description': 'Li3ClO test', 
    'options': {
        'max_wallclock_seconds': 1800, 
        'resources': {
            'num_machines': 1
        }
    }, 
    'label': 'Li3ClO test'
})


5. Construct the input dictionary. Note that this nesting is unique to the QuantumESPRESSO flavour of the FormationEnergy workchain.

In [ ]:
# If known, the dielctric constant can be directly provided as below. If not, it can be computed within the workchain by specifying the ph code
dielectric = orm.ArrayData()
dielectric.set_array('epsilon', np.array([[3.12, 0., 0.,], [0., 3.12, 0.], [0., 0., 3.12]]))

# Covariance matrix is needed to construct the gaussian charge model for the correction workchain.
cov_matrix = orm.ArrayData()
cov_matrix.set_array('sigma', np.eye(3))
inputs = {
    'relaxation_scheme': orm.Str('fixed'), # Run only scf calculation without relaxation
    # Structures
    'host_structure': host_structure,
    'defect_structure': defect_structure,
    'host_unitcell' : unitcell_structure,
    # Defect information 
    'defect_charge' : orm.Float(-3.0),
    'defect_site' : orm.List(list=defect_position),    # Position of the defect in crystal coordinates
    'chempot_sign': orm.Dict(dict={'Al':-1}), 
    'run_chem_pot_wc' : orm.Bool(False),
    'chemical_potential' : orm.Dict(dict={'Al':-524.03435, 'P':-191.03878}),
    'fermi_level' : orm.Float(0.0), # Fermi level is set to zero by default
    # Setup
    'run_pw_host' : orm.Bool(run_pw),
    'run_pw_defect_q0' : orm.Bool(run_pw),
    'run_pw_defect_q' : orm.Bool(run_pw),
    'run_v_host' : orm.Bool(run_v),
    'run_v_defect_q0' : orm.Bool(run_v),
    'run_v_defect_q' : orm.Bool(run_v),
    'run_rho_host' : orm.Bool(run_rho),
    'run_rho_defect_q0' : orm.Bool(run_rho),
    'run_rho_defect_q' : orm.Bool(run_rho),    
    'run_dfpt' : orm.Bool(False),
    # Method
    'correction_scheme' : orm.Str('gaussian'),
    'epsilon' : dielectric, # epsilon_inf = 3.2
    'cutoff' : orm.Float(400.0),
    'charge_model': {
        'model_type': orm.Str('fixed'),
        'fixed':{
            'covariance_matrix': cov_matrix            }
        # 'fitted': {
        #     'tolerance': orm.Float(1.0e-3),
        #     'strict_fit': orm.Bool(True),
        #     }
        },
    # Computational (chosen code is QE)
    'qe' : {
        'dft': {
            'supercell' : {
                'code' : pw_code,
                #'kpoints': kpoints, 
                'pseudopotential_family': pseudo_family, 
                # 'parameters' : pw_host_parameters,
                'scheduler_options' : pw_metadata,
                'settings' : pw_settings,

            },
            'unitcell' : {
                'code' : pw_code,
                #'kpoints': kpoints_unitcell, 
                'pseudopotential_family': pseudo_family, 
                #'parameters' : pw_parameters,
                'scheduler_options' : pw_metadata,
                'settings' : pw_settings,
            }            
        },
#         'dfpt' : {
#             'code' : ph_code,
#             'scheduler_options' : ph_metadata,
#         },
        'pp' : {
            'code' : pp_code,
            'scheduler_options' : pp_metadata,
        }
    }
}


6. Submit the workchain

In [ ]:
workchain_future = engine.submit(FormationEnergyWorkchainQE, **inputs)

7. Check the output

In [ ]:
if workchain_future.is_finished:
    if workchain_future.is_finished_ok:
        print('Finished successfully')
        print(
            "Workchain Formation energy (uncorrected): {} eV".format(
                workchain_future.outputs.formation_energy_uncorrected.value
            )
        )
        print(
            "Workchain Formation energy (corrected): {} eV".format(
                workchain_future.outputs.formation_energy_corrected.value
            )
        )    
        print(
            "Workchain Formation energy (corrected and aligned): {} eV".format(
                workchain_future.outputs.formation_energy_corrected_algned.value
            )
        )
    else:
        print('Excepted')
else:
    if workchain_future.is_excepted:
        print('Excepted')
    else:
        print('Not yet finished')


**Optional**
If the dielectric constant of the host materials is not know and you wish to compute it as part of the workchain, you have to specify the ph code as you did with pw and pp codes.

ph_code = orm.Code.get_from_string('ph@localhost')


ph_metadata = orm.Dict( dict={
    'description': 'Li3ClO test', 
    'options': {
        'max_wallclock_seconds': 1800, 
        'resources': {
            'num_machines': 12
        },
        'queue_name' : 'debug'
    }, 
    'label': 'Li3ClO test'
})